# BRAN2020 ARE 3D chunk

Date: 9 April, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

### BRAN2020 is order 50TB of `float32` data over nearly 9000 `netcdf` file assests in total.

#### required packages

In [1]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
import pandas as pd

In [2]:
xr.__version__

'2024.3.0'

#### ignore warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

#### start a local Dask client

In [4]:
import dask
from dask.distributed import Client, LocalCluster

cluster=LocalCluster(n_workers=28,processes=True)
client = Client(cluster)

2024-04-22 13:23:36,472 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,510 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,529 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,536 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,564 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,576 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,580 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,587 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,591 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,598 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,603 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,613 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,645 - distributed.nanny - WARNING - Restarting worker
2024-04-22 13:23:36,648 - distributed.

In [13]:
print(client)

<Client: 'tcp://127.0.0.1:46299' processes=28 threads=28, memory=251.18 GiB>


# ARD chunk workflow

## ARD - write zarr & chunk & write zarr

In [6]:
BRAN2020_ard_path = '/scratch/es60/ard/reanalysis/BRAN2020/ARD/'
ard_file_ID = 'BRAN2020-daily-mld-v04042024.zarr'

In [7]:
BRAN2020_daily = xr.open_zarr(BRAN2020_ard_path+ard_file_ID,consolidated=True)
BRAN2020_daily

<xarray.Dataset> Size: 241GB
Dimensions:   (Time: 11138, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 89kB 1993-01-01T12:00:00 ... 2023-06-30T1...
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    mld       (Time, yt_ocean, xt_ocean) float32 241GB dask.array<chunksize=(1, 1500, 3600), meta=np.ndarray>
Attributes: (12/13)
    NumFilesInSet:                   20
    acknowledgement:                 BRAN is made freely available by CSIRO B...
    catalogue_doi_url:               http://dx.doi.org/10.25914/6009627c7af03
    grid_tile:                       N/A
    grid_type:                       regular
    intake_esm_attrs:_data_format_:  netcdf
    ...                              ...
    intake_esm_attrs:source:         BRAN2020
    intake_esm_attrs:time_period:    daily
    intake_esm_attrs:variable:       mld
    intake_esm_dataset_key:          ocean.daily
    intake_esm_vars:                 ['mld']
    title:                           BRAN2020

## CHUNK for optomised monthly climatology = (27,1800,1500)

### "Consider either rechunking using `chunk()`, deleting or modifying `encoding['chunks']`"

In [8]:
def remove_zarr_encoding(DS):
    for var in DS:
        DS[var].encoding = {}

    for coord in DS.coords:
        DS[coord].encoding = {}
    return DS

In [9]:
ard_rcTime_file_ID = 'BRAN2020-daily-mld-chunk4month_clim-22042023.zarr'

In [10]:
BRAN2020_daily_rc =  BRAN2020_daily.chunk({'Time':27,'xt_ocean':1800,'yt_ocean':1500})
BRAN2020_daily_rc

<xarray.Dataset> Size: 241GB
Dimensions:   (Time: 11138, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 89kB 1993-01-01T12:00:00 ... 2023-06-30T1...
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    mld       (Time, yt_ocean, xt_ocean) float32 241GB dask.array<chunksize=(27, 1500, 1800), meta=np.ndarray>
Attributes: (12/13)
    NumFilesInSet:                   20
    acknowledgement:                 BRAN is made freely available by CSIRO B...
    catalogue_doi_url:               http://dx.doi.org/10.25914/6009627c7af03
    grid_tile:                       N/A
    grid_type:                       regular
    intake_esm_attrs:_data_format_:  netcdf
    ...                              ...
    intake_esm_attrs:source:         BRAN2020
    intake_esm_attrs:time_period:    daily
    intake_esm_attrs:variable:       mld
    intake_esm_dataset_key:          ocean.daily
    intake_esm_vars:                 ['mld']
    title:                           BRAN2020

In [11]:
BRAN2020_daily_rc = remove_zarr_encoding(BRAN2020_daily_rc)

In [12]:
%%time
BRAN2020_daily_rc.to_zarr(BRAN2020_ard_path+ard_rcTime_file_ID,consolidated=True)
!touch /scratch/es60/ard/reanalysis/BRAN2020/ARD/logs/finished_mld_clim_month_chunk_22042024.log

CPU times: user 1min 6s, sys: 7.29 s, total: 1min 14s
Wall time: 1min 33s


## end ARD chunk workflow $\Uparrow$

# $The$ $End$

In [ ]:
client.shutdown()